In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys
sys.path.append('../../')

In [18]:
print(sys.path)

['/home/d_wang/nlp/MAF/src/baselines', '/home/d_wang/anaconda3/envs/maf/lib/python310.zip', '/home/d_wang/anaconda3/envs/maf/lib/python3.10', '/home/d_wang/anaconda3/envs/maf/lib/python3.10/lib-dynload', '', '/home/d_wang/anaconda3/envs/maf/lib/python3.10/site-packages', '../../']


: 

In [4]:
import asyncio
import math
import os
import time
from typing import List
import pandas as pd
import torch
from tqdm import tqdm
from src.utils import Prompt, acall_gpt, call_gpt, VICUNA_MODEL_PATH, ALPACA_MODEL_PATH, get_gpu_memory
from fastchat.model.model_adapter import get_conversation_template
from fastchat.serve.inference import load_model
from src.gsm_maf.task_init import OSInit
import baseline_utils
from types import SimpleNamespace
import json


/home/d_wang/anaconda3/envs/maf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
args = {
    'data_dir': '../../data/gsm_data',
    'save_dir': 'models',
    'debug': False,
    'exp_label': 'default',
    'task': 'pot_gsm',
    'model': 'vicuna',
    'max_tokens': 2048,
    'temperature': 0.0,
}
args['ckpt_path'] = os.path.join(args['save_dir'], args['exp_label'])
args = SimpleNamespace(**args)

In [16]:
def gsm_baseline(model, task):
    model.setup_prompt_from_examples_file(f'../../prompt/os_baseline/{task}.txt')
    
    for i in range(3):
        for variant in ['original', 'irc']:
            data = baseline_utils.load_gsm_data(os.path.join(args.data_dir, f'gsmic_mixed_{i}_{variant}.jsonl'))
            if (os.path.exists(os.path.join(args.save_dir, f'gsmic_mixed_{i}_{variant}_output_{args.model}_{task}.json')) 
                or os.path.exists(os.path.join(args.save_dir, f'hand_gsmic_mixed_{i}_{variant}_output_{args.model}_{task}.json'))):
                continue
            output_file = os.path.join(args.save_dir, f'gsmic_mixed_{i}_{variant}_output_{args.model}_{task}.json')
            batch_size = 5
            for j in range(0, len(data), batch_size):
                batch = data[j:j+batch_size]
                batch_solutions = model([d['input'] for d in batch])
                for k in range(len(batch)):
                    data[j+k]['output'] = batch_solutions[k]
                print(data[j : j+batch_size])
                with open(output_file, 'w') as f:
                  f.write(json.dumps(data) + '\n')
                time.sleep(1)
            

In [14]:
os_task_init = OSInit(
        prompt_examples=f'../../prompt/os_baseline/pot_gsm.txt',
        engine=args.model,
        temperature=0.01,
)

0,1,2


Loading checkpoint shards: 100%|██████████| 3/3 [00:23<00:00,  7.73s/it]


In [17]:

print(args.model, args.task)
gsm_baseline(os_task_init, args.task)

vicuna pot_gsm
query: A chat between a curious human and an artificial intelligence assistant. The assistant gives a single helpful and detailed python function to solve each of the human's math questions. 

USER:
# Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
ASSISTANT:
# solution using Python:

def solution():
    """Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?"""
    jason_lollipops_initial = 20
    jason_lollipops_after = 12
    denny_lollipops = jason_lollipops_initial - jason_lollipops_after
    result = denny_lollipops
    return result


USER:
# Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
ASSISTANT:
# solution using Python:

def solution():
    """There are 15 trees in the grove. 

100%|██████████| 5/5 [04:36<00:00, 55.26s/it]


[{'input': "Oli's banana split has 4 scoops of ice cream while Victoria has twice more scoops of ice cream than Oli's. How many more scoops of ice cream does Victoria have than Oli?", 'target': '4', 'n_steps': 2, 'output': 'def solution():\nif scoops\\_of\\_ice\\_cream\\_oli == 4:\nscoops\\_of\\_ice\\_cream\\_victoria = 8\nprint("Oli\'s banana split has 4 scoops of ice cream while Victoria has twice more scoops of ice cream than Oli\'s. Victoria has 8 more scoops of ice cream than Oli.")\nelif scoops\\_of\\_ice\\_cream\\_oli > 4:\nscoops\\_of\\_ice\\_cream\\_victoria = scoops\\_of\\_ice\\_cream\\_oli \\* 2\nprint("Oli\'s banana split has more than 4 scoops of ice cream while Victoria has twice more scoops of ice cream than Oli\'s. Victoria has {} more scoops of ice cream than Oli.".format(scoops\\_of\\_ice\\_cream\\_victoria - 4))\nelse:\nscoops\\_of\\_ice\\_cream\\_victoria = scoops\\_of\\_ice\\_cream\\_oli \\* 2\nprint("Oli\'s banana split has 4 scoops of ice cream while Victoria has

100%|██████████| 5/5 [03:44<00:00, 44.93s/it]


[{'input': 'There are one hundred tickets to be sold for a volleyball game. Andrea sold twice as many tickets as Jude while Sandra sold 4 more than half the number of tickets Jude sold. If Jude sold 16 tickets, how many tickets need to be sold?', 'target': '40', 'n_steps': 5, 'output': 'def solution():\n"""There are one hundred tickets to be sold for a volleyball game. Andrea sold twice as many tickets as Jude while Sandra sold 4 more than half the number of tickets Jude sold. If Jude sold 16 tickets, how many tickets need to be sold?"""\ntickets_total = 100\ntickets_jude = 16\ntickets_andrea = tickets_total \\* 2\ntickets_jude = tickets_andrea - tickets_jude\ntickets_sandra = (tickets_total \\* 3) / 2 + 4\ntickets_needed = tickets_sandra - tickets_jude\nresult = tickets_needed\nreturn result\n\nPlease let me know if you have any questions or if there is anything else I can help you with.'}, {'input': 'A cake of 400 grams is divided into eight equal parts. Nathalie eats one-eighth of t

...

[{'input': 'Jennifer decides to share her sweets between herself and her 3 friends. She has 212 green sweets, 310 blue sweets and 502 yellow sweets. How many sweets will Jennifer and her friends get each?', 'target': '256', 'n_steps': 2, 'output': 'def solution():\n"""Jennifer decides to share her sweets between herself and her 3 friends. She has 212 green sweets, 310 blue sweets and 502 yellow sweets. How many sweets will Jennifer and her friends get each?"""\n\n# Calculate total number of sweets\ntotal_sweets = 212 + 310 + 502\n\n# Calculate number of sweets for Jennifer and her friends\njennifer_friends = total_sweets / 4\n\n# Calculate number of sweets for each friend\nsweets_per_friend = total_sweets - (jennifer_friends * 4)\n\n# Print the result\nprint("Jennifer and her friends will get", jennifer_friends, "sweets each.")\nprint("Each friend will get", sweets_per_friend, "sweets.")\n\nreturn jennifer\\_friends, sweets\\_per\\_friend\n\njennifer\\_friends, sweets\\_per\\_friend = 

100%|██████████| 5/5 [05:21<00:00, 64.20s/it]


[{'input': "Angela wants to check her math homework answers with her friends, but some of them aren't done yet. Out of 20 problems, Martha has finished 2, Jenna has finished four times the number Martha did minus 2, and Mark has finished half the number Jenna did. If none of the friends worked on any of the same problems, how many problems have no one but Angela finished?", 'target': '9', 'n_steps': 5, 'output': 'def solution():\n"""Angela wants to check her math homework answers with her friends, but some of them aren\'t done yet. Out of 20 problems, Martha has finished 2, Jenna has finished four times the number Martha did minus 2, and Mark has finished half the number Jenna did. If none of the friends worked on any of the same problems, how many problems have no one but Angela finished?"""\nnum_problems = 20\nnum_martha_finished = 2\nnum_jenna_finished = 4 \\* (num\\_martha\\_finished - 2)\nnum\\_mark\\_finished = (num\\_jenna\\_finished / 2) \\* 2\nnum\\_problems\\_no\\_one\\_finis

 80%|████████  | 4/5 [03:28<00:52, 52.10s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 print(args.model, args.task)                                                                 │
│ ❱ 2 gsm_baseline(os_task_init, args.task)                                                        │
│   3                                                                                              │
│                                                                                                  │
│ in gsm_baseline:14                                                                               │
│                                                                                                  │
│   11 │   │   │   batch_size = 5                                                                  │
│   12 │   │   │   for j in range(0, len(data), batch_size):                                       │
│   13 │   │   │   │   batch = data[j:j+batch_size]                                                │
│ ❱ 14 │   │   │   │   batch_solutions = model([d['input'] for d in batch])                        │
│   15 │   │   │   │   for k in range(len(batch)):                                                 │
│   16 │   │   │   │   │   data[j+k]['output'] = batch_solutions[k]                                │
│   17 │   │   │   │   print(data[j : j+batch_size])                                               │
│                                                                                                  │
│ /home/d_wang/nlp/MAF/src/baselines/../../src/gsm_maf/task_init.py:163 in __call__                │
│                                                                                                  │
│   160 │   │   │   input_ids = self.tokenizer([generation_queries[i]]).input_ids                  │
│   161 │   │   │   input_ids = torch.as_tensor(input_ids).to(self.model.device)                   │
│   162 │   │   │   with torch.no_grad():                                                          │
│ ❱ 163 │   │   │   │   output_ids = self.model.generate(                                          │
│   164 │   │   │   │   │   input_ids,                                                             │
│   165 │   │   │   │   │   do_sample=True,                                                        │
│   166 │   │   │   │   │   temperature=self.temperature,                                          │
│                                                                                                  │
│ /home/d_wang/anaconda3/envs/maf/lib/python3.10/site-packages/torch/utils/_contextlib.py:115 in   │
│ decorate_context                                                                                 │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /home/d_wang/anaconda3/envs/maf/lib/python3.10/site-packages/transformers/generation/utils.py:14 │
│ 85 in generate                                             